In [1]:
from time import sleep, strftime
from random import randint
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import smtplib
from email.mime.multipart import MIMEMultipart
from urllib.parse import quote
from datetime import datetime

In [2]:
# Load more results to maximize the scraping

def load_more():
    try:
        # more_results = '//a[@class = "moreButton"]'
        plustard = '//a[@label="Touver des trains plus tard"]'
        driver.find_element_by_xpath(plustard).click()
        print('sleeping.....')
        sleep(randint(25,35))
    except:
        pass
    
    # Conversão das datas para o formato codificado
def encode_date(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%d %H:%M:%S")
    date_iso = date_obj.isoformat()  # Converte para formato ISO 8601
    return quote(date_iso)  # Codifica para URL

In [3]:

def page_scrape():
    
    # getting the prices
    xp_prices = '//div[@data-test="is-cheapest-tag"]/span'
    xp_prices = '(//span[@data-test="is-cheapest-tag"]/preceding-sibling::div//span)[1]'
    prices = driver.find_elements_by_xpath(xp_prices)
    prices_list = [price.text.replace('&nbsp;€','') for price in prices if price.text != '']
    prices_list = list(map(int, prices_list))
    
    print(prices_list)

In [26]:
def start_trainline(city_from, city_to, inward_date, outward_date):
    
    service = Service()

    # options = webdriver.ChromeOptions()
    # options.add_argument("--incognito")

    # driver = webdriver.Chrome(service=service, options=options)
    # sleep(2)
    
    encoded_outward_date = encode_date(outward_date)
    encoded_inward_date = encode_date(inward_date)
    
    options = webdriver.EdgeOptions()
    options.use_chromium = True
    options.add_argument("-inprivate")
    
    driver = webdriver.Edge(service=service, options = options)
    sleep(2)
    
    trainline_url = (
    "https://www.thetrainline.com/book/results?"
    + f"journeySearchType=return&origin={city_from}"
    + f"&destination={city_to}&outwardDate={encoded_outward_date}"
    + "&outwardDateType=departAfter"
    + f"&inwardDate={encoded_inward_date}&inwardDateType=departAfter"
    + "&selectedTab=train&splitSave=true&lang=fr"
    + "&transportModes%5B%5D=mixed"
    + "&dpiCookieId=HCEAT7USDRAJWSS08WIJINST5&partnershipType=accommodation"
    + "&partnershipSelection=false&selectedOutward=6nVfyGbH7RU%3D%3A9yGUtSZSXHQ%3D%2CbUuQ3iTP8Ec%3D%3AStandard"
)
    
    print(trainline_url)
    
    driver.get(trainline_url)
    sleep(randint(20,30))
    
    print('cloasing cookies.....')
    # # sometimes a popup shows up, so we can use a try statement to check it and close
    try:
        # xp_popup_close = '//button[contains(@id,"dialog-close") and contains(@class,"Button-No-Standard-Style close ")]'
        xp_popup_close = '//button[contains(@class, "onetrust-close-btn-handler banner-close-button ot-close-link)]'
        driver.find_elements_by_xpath(xp_popup_close)[5].click()
    except Exception as e:
        print("sem cookies")
        pass
    sleep(randint(60,95))
    
    print('loading more.....')
    
    # load_more()
    try:
        # more_results = '//a[@class = "moreButton"]'
        plustard = '//a[@label="Touver des trains plus tard"]'
        driver.find_element_by_xpath(plustard).click()
        print('sleeping.....')
        sleep(randint(25,35))
    except:
        pass
    
    # page_scrape()
    # getting the prices
    xp_prices = '//div[@data-test="is-cheapest-tag"]/span'
    xp_prices = '(//span[@data-test="is-cheapest-tag"]/preceding-sibling::div//span)[1]'
    price = driver.find_element(By.XPATH, xp_prices)
    price_text = price.text.replace(" €", "").replace(",", ".")
    price_float = float(price_text)
    df = pd.DataFrame({"Preço (€)": [price_float],
                       "Data Ida": [outward_date] })
    
    print(df)
    
    

In [ ]:
origin_code = "urn%3Atrainline%3Ageneric%3Aloc%3A5601"  # Código Rouen
destination_code = "urn%3Atrainline%3Ageneric%3Aloc%3A7480"  # Código Munchen
outward_date = "2025-02-17 06:00:37"  # Data de ida (legível)
inward_date = "2025-02-20 06:00:37"  # Data de volta (legível)

start_trainline(origin_code, destination_code, inward_date, outward_date)

https://www.thetrainline.com/book/results?journeySearchType=return&origin=urn%3Atrainline%3Ageneric%3Aloc%3A5601&destination=urn%3Atrainline%3Ageneric%3Aloc%3A7480&outwardDate=2025-02-17T06%3A00%3A37&outwardDateType=departAfter&inwardDate=2025-02-20T06%3A00%3A37&inwardDateType=departAfter&selectedTab=train&splitSave=true&lang=fr&transportModes%5B%5D=mixed&dpiCookieId=HCEAT7USDRAJWSS08WIJINST5&partnershipType=accommodation&partnershipSelection=false&selectedOutward=6nVfyGbH7RU%3D%3A9yGUtSZSXHQ%3D%2CbUuQ3iTP8Ec%3D%3AStandard
cloasing cookies.....
sem cookies
